# Agentcore Inbound Auth

AgentCore Identity lets you validate inbound access (Inbound Auth) for users and applications calling agents or tools in an AgentCore Runtime or validate access to AgentCore Gateway targets. It also provide secure outbound access (Outbound Auth) from an agent to external services or a Gateway target. It integrates with your existing identity providers (such as Amazon Cognito) while enforcing permission boundaries for agents acting independently or on behalf of users (via OAuth).

Inbound Auth validates callers attempting to invoke agents or tools, whether they're hosted in AgentCore Runtime, AgentCore Gateway , or in other environments. Inbound Auth works with IAM (SigV4 credentials) or with OAuth authorization.

By default, Amazon Bedrock AgentCore uses IAM credentials, meaning user requests to the agent are authenticated with the user's IAM credentials. If you use OAuth, you will need to specify the following when configuring your AgentCore Runtime resources or AgentCore Gateway endpoints:

- OAuth discovery server Url — A string that must match the pattern ^.+/\.well-known/openid-configuration$ for OpenID Connect discovery URLs

- Allowed audiences — List of allowed audiences for JWT tokens

- Allowed clients — List of allowed client identifiers

Before the user can call the agent, the client application must have the user authenticate with the OAuth authorizer. Your client receives a bearer token which it then passes to the agent in an invocation request. Upon receipt the agent validates the token with the authorization server before allowing access.

## About this Notebook

In this notebook, we will modify the mortgage assistant agent you deployed in 04-agentcore-gateway and configure it for Inbound Auth using Cognito as the Identity Provider. You will set up a Cognito User pool with one user and an app client. You will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime with Inbound Auth using the Cognito user pool. 

## Architecture

![Inbound Auth Architecture](../images/inbound-auth.png)

## Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime with Inbound Auth using Amazon Cognito

## Setting up Amazon Cognito for Authentication

Lets provision a Cognito Userpool with an App client and one test user. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. To do so, we will use the `setup_cognito_user_pool` support function from our `utils` script.

In [ ]:
import utils

cognito_config = utils.setup_cognito_user_pool()

user_pool_id = cognito_config['pool_id']
discovery_url = cognito_config['discovery_url']
client_id = cognito_config['client_id']
bearer_token = cognito_config['bearer_token']

print(f"user_pool_id: {user_pool_id}")
print(f"discovery_url: {discovery_url}")
print(f"client_id: {client_id}")
print(f"bearer_token: {bearer_token}")

## Preparing your agent for deployment on AgentCore Runtime

Let's start with our Strands Agent we created in the 04-agentcore-gateway and configure it with Inbound Auth that uses Amazon Cognito as the Identity Provider.

### Configure AgentCore Runtime

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

**Important** - Update the Cognito Discovery url and the Cognito App client id from the previous steps.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "mortgage_assistant"
agentcore_gateway_iam_role = utils.create_agentcore_role(agent_name)

response = agentcore_runtime.configure(
    entrypoint="mortgage_agent_runtime_gw.py",
    execution_role=agentcore_gateway_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response

### Launching agent to AgentCore Runtime

Now let's launch the agent to the AgentCore Runtime. 

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
launch_result

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime without authorization

Finally, we can invoke our AgentCore Runtime with a payload. Try running the following cell and we expect to see an error that says 

**"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type".**

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are the benefits of a 15-year mortgage?"})
invoke_response

### Invoking AgentCore Runtime with authorization

Lets invoke the agent with the right authorization token type. In our case, it will be the Cognito access token. 

In [ ]:
import json

def print_response_text(invoke_response):
    response_bytes=invoke_response['response']
    response_strings = [item for item in response_bytes]
    # print(response_bytes)
    response_combined_string = "".join(response_strings)
    # print(response_combined_string)
    
    response_dict = json.loads(response_combined_string)
    response_text=response_dict["result"]["content"][0]["text"]
    print(response_text)

In [ ]:
bearer_token = utils.reauthenticate_user(client_id)

invoke_response = agentcore_runtime.invoke(
    {"prompt": "what is the credit score for me? my customer id is 1234"}, 
    bearer_token=bearer_token
)

In [ ]:
print_response_text(invoke_response)

### 🎉 Congratulation! 
You have added inbound authentication to your multi-agent Mortgage Assistant. 